In [ ]:
import pandas as pd
import numpy as np

verified_data_file_path = "../verified_data_01_15_2024.xlsx"
df_verified = pd.read_excel(verified_data_file_path)
df_verified.head()

# VoteGenie

In [ ]:
from genie_master import GenieMaster

genie_db_path = "./chroma_qadata_db/"
gm = GenieMaster(db_path=genie_db_path)
print("Genie is ready...", gm.model_is_ready())
print("Document collection count...", gm._document_count())

In [ ]:
# # make a list of genies for each unique politician
# names = df_verified.name.unique()
# genies = {n: gm.get_genie(n) for n in names}

In [ ]:
# def process_row(row):
#     output = genies[row["name"]].ask(row["question"])
#     result = output["result"]
#     row['prediction'] = result.get('answer', "")
#     row['reasoning'] = result.get('reasoning', "")
#     row['evidence'] = result.get('evidence', "")
#     row['cost'] = output['total_cost']

#     print("NAME:", row["name"])
#     print("QUESTION:", row["question"])
#     print("ANSWER:", row["stance"])
#     print("PREDICTION:", row["prediction"])
#     print("-"*30)


# df_results = df_verified.apply(process_row, axis=1)
# df_results.head()

In [ ]:
from vote_easy_genie_lib import get_df_results
output_csv = "temp/temp.csv"
output_xlsx = "verify_result.xlsx"
df_results = await get_df_results(df_verified, vectorstore_path=genie_db_path, output_csv=output_csv, output_xlsx=output_xlsx, model_name="gpt-4-1106-preview")

In [ ]:
def check_answer(df, prediction_col_name, verification_col_name="stance"):
    """returns a list where if ans and pred is the same, True; if ans and pred is 
    different, False"""
    df[prediction_col_name] = df[prediction_col_name].str.lower()
    df[verification_col_name] = df[verification_col_name].str.lower()
    return np.where((df[prediction_col_name] == df[verification_col_name]), True, False)
prediction_col_name="answer"
df_incorrect_entries = df_results.loc[~check_answer(df_results, prediction_col_name=prediction_col_name)]
print(f"Accuracy: {df_results.shape[0]-df_incorrect_entries.shape[0]}/{df_results.shape[0]}")
print(f"Percentage accuracy: {1-df_incorrect_entries.shape[0]/df_results.shape[0]}")

In [ ]:
prediction_col_name="answer"
for _, row in df_incorrect_entries.iterrows():
    print("NAME:", row["name"])
    print("QUESTION:", row["question"])
    print("Prediction\tAnswer")
    print(f"{row[prediction_col_name]}\t\t{row['stance']}")
    print(f"Reasoning\t{row['reasoning']}")
    print(f"Evidence\t{row['evidence']}")
    print(f"Source\t\t{row['source_content']}")
    print("\n")

In [ ]:
for _, row in df_results.loc[check_answer(df_results, prediction_col_name=prediction_col_name)].iterrows():
    print("NAME:", row["name"])
    print("QUESTION:", row["question"])
    print("Prediction\tAnswer")
    print(f"{row[prediction_col_name]}\t\t{row['stance']}")
    print(f"Reasoning\t{row['reasoning']}")
    print(f"Evidence\t{row['evidence']}")
    print(f"Source\t\t{row['source_content']}")
    print("\n")

# RAGAS

In [ ]:
import pandas as pd
df = pd.read_excel("verify_result.xlsx")
df.head(3)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

In [ ]:
from ragas import evaluate
df_temp = df.loc[:3]
result = evaluate(
    df,
    metrics=[
       answer_relevancy,
        faithfulness,
        context_recall,
        context_precision 
    ],
    column_map={
        ""
    }
)

result

In [ ]:
import pandas as pd
df = pd.read_excel("verify_result.xlsx")
df.head(3)

In [ ]:
name = "Joe Biden"
questions = df.question[:5].to_list()
questions

In [ ]:
from genie_master import GenieMaster

genie_db_path = "./chroma_qadata_db/"
gm = GenieMaster(db_path=genie_db_path)
print("Genie is ready...", gm.model_is_ready())
print("Document collection count...", gm._document_count())

In [ ]:
import time
genie = gm.get_genie(name)
print(genie)

In [ ]:
start_time = time.time()
results_1 = list()
for q in questions[:1]:
    results_1.append(genie.ask(q))
    print("Question answered")
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
results_1[0]

In [ ]:
start_time = time.time()
results_2 = genie.batch_ask(questions)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
results_2['results'][0]

In [ ]:
start_time = time.time()
results_3 = await genie.async_batch_ask(questions)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
results_3

In [ ]:
start_time = time.time()
base_batch_result = genie.base_batch_ask(questions)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# RAGAS evaluation
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas import evaluate

In [ ]:
base_batch_result

In [ ]:
import pandas as pd
import json
from datasets import Dataset

In [ ]:
data_samples

In [ ]:
data_samples = {
    "question": questions,
    "contexts": [[doc.page_content for doc in r["context"]] for r in base_batch_result],
    "answer": [json.dumps(r["result"]) for r in base_batch_result],
    "ground_truths": [["yes"]*4]*5
}
dataset = Dataset.from_dict(data_samples)
dataset

In [ ]:
dataset.to_pandas()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
evaluate(
    dataset,
    metrics=[
        # context_precision,
        faithfulness,
        # answer_relevancy,
        # context_recall,
    ],

)

In [ ]:
data_samples

In [ ]:
data_sample_2 = base_batch_result.copy()
data_sample_2[0]

In [ ]:
for entry in data_sample_2:
    entry["source_documents"] = entry.pop("context")
    entry["query"] = entry.pop("question")
    entry["result"] = json.dumps(entry["result"])
data_sample_2

In [ ]:
# eval = {
#     m.name: RagasEvaluatorChain(metric=m).evaluate(
#         questions,
#         data_sample_2
#     )
#     for m in [
#         faithfulness,
#         answer_relevancy,
#         context_precision,
#         context_recall,
#     ]
# }
eval = RagasEvaluatorChain(metric=faithfulness)(data_sample_2[0])
eval

In [1]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

Generating baseline split:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
result = evaluate(
    fiqa_eval["baseline"].select(range(3)), # selecting only 3
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)
result

evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:40<00:00, 40.70s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:08<00:00,  8.68s/it]


{'context_precision': 0.3333, 'faithfulness': 0.9444, 'answer_relevancy': 0.9756, 'context_recall': 0.9524}